In [15]:
import os, sys
import logging, logging.config
from tqdm import tqdm
import glob

import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import shape

import numpy as np

import misc_fct

In [34]:
TILES_DIR='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/*.tif'
ROADS='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/non_forest_wide_roads.shp'

roads=gpd.read_file(ROADS)

In [27]:
files=glob.glob(TILES_DIR)
print(files[:5])

['/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/17_68345_46241.tif', '/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/17_68381_46172.tif', '/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/17_68369_46214.tif', '/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/17_68399_46269.tif', '/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images-256/17_68342_46196.tif']


In [29]:
geom=[]

for file in tqdm(files, desc='Checking files'):
    with rasterio.open(file) as f:
        image = f.read(1)

        lim_sup=200
        lim_inf=1

        # create a binary image, 0 where there's nodata, 1 where it's valid
        is_valid = ((image < lim_inf) | (image > lim_sup)).astype(np.uint8)

        
        # vectorize the binary image, supplying the transform so it returns maps coords
        for coords, value in rasterio.features.shapes(is_valid, transform=f.transform):

            # ignore polygons corresponding to nodata
            if value != 0:
                # convert geojson to shapely geometry
                geom.append(shape(coords))

fid=[x for x in range(1, len(geom)+1)]
zones={'id':fid, 'geometry': geom}


In [30]:
extrem_zones=gpd.GeoDataFrame(zones, crs='EPSG:3857')

In [36]:
roads_reproject=roads.to_crs(epsg=3857)

misc_fct.test_crs(roads_reproject.crs, extrem_zones.crs)

extrem_zones_on_roads=gpd.overlay(extrem_zones,roads_reproject)

In [38]:
extrem_zones_on_roads.to_file('/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/test_extrem_pixels.shp')